In [171]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
from sklearn.datasets import load_diabetes, load_digits
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor, \
    Lasso, ElasticNet, Lars, LassoLars, OrthogonalMatchingPursuit, \
    BayesianRidge, ARDRegression, \
    RANSACRegressor, TheilSenRegressor, HuberRegressor, \
    LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split

InteractiveShell.ast_node_interactivity = "all" # allow multiple cell output

In [172]:
# Import diabetes & digits data as examples
diabetes_X, diabetes_y = load_diabetes(return_X_y=True) #diabetes for regression
digits_X, digits_y = load_digits(return_X_y=True) #digits for classification

# Train-Test split
diabetes_X_train, diabetes_X_test, diabetes_y_train, diabetes_y_test = train_test_split(diabetes_X, diabetes_y, test_size=0.2, random_state=42)
digits_X_train, digits_X_test, digits_y_train, digits_y_test = train_test_split(digits_X, digits_y, test_size=0.2, random_state=42)

Linear Models
============

As we know, scikit-learn *Estimator* object have a `fit` method, to learn from the model, used on the training data. It also have a `predict` method, usually used on test data after the training step, to predict target based on $X$ features. <br>

It also have a `score` method, to evaluate the performance of the model for the task.  
For a regression task, the metric used is usually the $R^{2}$, defined as follow:

$$R^{2} = 1 - \frac{\text{Residual Sum of Squares}}{\text{Total Sum of Squares}}$$
where $\text{Residual Sum of Squares} = \sum{(y_{true} - y_{pred})^{2}}$ and $\text{Total Sum of Squares} = \sum{(y_{true} - \frac{\sum_{1}^{n}y_{true}}{n})^{2}}$.

The best possible score is 1 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of $y$, disregarding the input features, would get a $R^{2}$ score of 0.

For a classification task, it's the mean accuracy:
$$ \frac{\text{# good prediction}}{n_{samples}}$$

*Estimator* can also have a `set_params` and a `get_params` depending on the algorithm.

# 1. Regression

The following are a set of methods intended for regression in which the target value, $\hat{y}$,  is expected to be a linear combination of the $p$ features $x_i$:

$$\hat{y}(w, x) = w_0 + w_1 x_1 + ... + w_p x_p$$  
where $w = (w_1,..., w_p)$ are the `coef_` and $w_0$ the `intercept_`

In [173]:
# Store intercept, weights and score metric to compare models

reg_keys = ["R^2", "intercept"] + [f"w{i}" for i in range(diabetes_X_train.shape[1])]
reg_results = []
reg_index = []

## 1.1 Classical linear regressors

### 1.1.1 Ordinary Least Squares

LinearRegression fits a linear model with coefficients $w = (w_1,..., w_p)$ to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation: 
$$\min_{w} || X w - y||_2^2$$

To sum up, *OLS* is a strategy to obtain, from your model, a ‘straight line’ which is as close as possible to your data points. Even though *OLS* is not the only optimization strategy, it is the most popular for this kind of tasks, since the outputs of the regression (that are, coefficients) are unbiased estimators of the real values of `alpha` ($w_0$) and `beta`$(w_1,..., w_p)$. Indeed, according to the Gauss-Markov Theorem, under some assumptions of the linear regression model (linearity in parameters, random sampling of observations, conditional mean equal to zero, absence of multicollinearity, homoskedasticity of errors), the *OLS* estimators `α` and `β` are the Best Linear Unbiased Estimators (BLUE) of the real values of `α` and `β`.

<img src="../resources/img/ols.png" width="400">

LinearRegression will take in its `fit` method arrays $X$, $y$ and will store the coefficients  of the linear model in its `coef_` member.

In [174]:
ols = LinearRegression()
ols.fit(diabetes_X_train, diabetes_y_train)

ols_coef = ols.coef_
ols_intercept = ols.intercept_

ols_score = ols.score(diabetes_X_test, diabetes_y_test)

print(f"Intercept: {ols_intercept}")
print(f"Coefficients: {ols_coef}")
print(f"R^2: {ols_score}")

LinearRegression()

Intercept: 151.3456553477407
Coefficients: [  37.90031426 -241.96624835  542.42575342  347.70830529 -931.46126093
  518.04405547  163.40353476  275.31003837  736.18909839   48.67112488]
R^2: 0.45260660216173776


In [175]:
reg_index.append("OLS")
reg_results.append(dict(zip(reg_keys, [ols_score, ols_intercept] + ols_coef.tolist())))

The coefficient estimates for *OLS* rely on the **independence of the features**. When features are correlated and the columns of $X$ have an approximate linear dependence, $X$ becomes close to singular and as a result, the least-squares estimate becomes highly sensitive to random errors in the observed target, producing a large variance. This situation of multicollinearity can arise, for example, when data are collected without an experimental design.

<u>Complexity</u>: The least squares solution is computed using the singular value decomposition of $X$. If $X$ is a matrix of shape `(n_samples, n_features)` this method has a cost of $O(n_{\text{samples}} n_{\text{features}}^2)$, assuming that $n_{\text{samples}} \geq n_{\text{features}}$.

[OLS explained visually](https://setosa.io/ev/ordinary-least-squares-regression/)

*Ridge* and *Lasso* regression are some of the simple techniques to reduce model complexity and prevent over-fitting which may result from simple linear regression.

### 1.1.2 Ridge Regression

*Ridge* regression addresses some of the problems of *OLS* by imposing a **penalty on the size of the coefficients. Then it shrinks the coefficients and helps to reduce the model complexity and multi-collinearity**.<br>
The ridge coefficients minimize a penalized residual sum of squares:
$$\min_{w} || X w - y||_2^2 + \alpha ||w||_2^2$$
The complexity parameter $\alpha \geq 0$ controls the amount of shrinkage: the larger the value of , the greater the amount of shrinkage and thus the coefficients become more robust to collinearity.

<img src="../resources/img/ridge.png" width="450">

<u>Complexity</u>: This method has the same order of complexity as OLS

## 1.2 Regressors with variable selection

The following estimators have built-in variable selection fitting procedures, but any estimator using a $\ell_1$ or *elastic-net* penalty also performs variable selection: typically *SGD* with an appropriate penalty.

### 1.2.1 Lasso Regression

The *Lasso* is a linear model that estimates sparse coefficients. It is useful in some contexts due to its **tendency to prefer solutions with fewer non-zero coefficients, effectively reducing the number of features** upon which the given solution is dependent. For this reason Lasso and its variants are fundamental to the field of compressed sensing. <br>
It consists of a linear model with an added regularization term. The objective function to minimize is:
$$\min_{w} { \frac{1}{2n_{\text{samples}}} ||X w - y||_2 ^ 2 + \alpha ||w||_1}$$

The lasso estimate thus solves the minimization of the least-squares penalty with $\alpha ||w||_1$ added, where $\alpha ||w||_1$ is a constant and $||w||_1$ is the $\ell_1$-norm of the coefficient vector.<br>
The implementation in the class Lasso uses coordinate descent as the algorithm to fit the coefficients. <br>
The `alpha` parameter controls the degree of sparsity of the estimated coefficients.

In [176]:
lasso = Lasso(alpha=0.1)
lasso.fit(diabetes_X_train, diabetes_y_train)

lasso_coef = lasso.coef_
lasso_intercept = lasso.intercept_

lasso_score = lasso.score(diabetes_X_test, diabetes_y_test)

print(f"Intercept: {lasso_intercept}")
print(f"Coefficients: {lasso_coef}")
print(f"R^2: {lasso_score}")

Lasso(alpha=0.1)

Intercept: 151.57488311231123
Coefficients: [   0.         -152.66706552  552.6941724   303.37055083  -81.3648345
   -0.         -229.25829816    0.          447.91818931   29.64235375]
R^2: 0.4718552616908692


In [177]:
reg_index.append("Lasso")
reg_results.append(dict(zip(reg_keys, [lasso_score, lasso_intercept] + lasso_coef.tolist())))

As the Lasso regression yields sparse models, it **not only helps in reducing over-fitting but it can help us in feature selection**.

The equivalence between `alpha` and the regularization parameter of *SVM*, `C` is given by $\alpha = \frac{1}{C}$ or $\alpha = \frac{1}{C \cdot n_{samples}}$ depending on the estimator and the exact objective function optimized by the model.

### 1.2.2 Elastic-Net

ElasticNet is a linear regression model trained with both $\ell_1$ and $\ell_2$-norm regularization of the coefficients. This combination allows for learning a sparse model where few of the weights are non-zero like **Lasso**, while still maintaining the regularization properties of **Ridge**. We control the convex combination of $\ell_1$ and $\ell_2$ using the `l1_ratio` parameter.

*Elastic-net* is useful when there are multiple features which are correlated with one another. Lasso is likely to pick one of these at random, while *elastic-net* is likely to pick both.<br>
A practical advantage of trading-off between *Lasso* and *Ridge* is that it allows *Elastic-Net* to inherit some of *Ridge*’s stability under rotation.<br>
The objective function to minimize is in this case:
$$\min_{w} { \frac{1}{2n_{\text{samples}}} ||X w - y||_2 ^ 2 + \alpha \rho ||w||_1 +
\frac{\alpha(1-\rho)}{2} ||w||_2 ^ 2}$$

In [178]:
elastic_net = ElasticNet()
elastic_net.fit(diabetes_X_train, diabetes_y_train)

elastic_coef = elastic_net.coef_
elastic_intercept = elastic_net.intercept_

elastic_score = elastic_net.score(diabetes_X_test, diabetes_y_test)

print(f"Intercept: {elastic_intercept}")
print(f"Coefficients: {elastic_coef}")
print(f"R^2: {elastic_score}")

ElasticNet()

Intercept: 153.71775135185501
Coefficients: [ 0.40100381  0.          3.41353374  2.32617327  0.45957655  0.1255522
 -1.78311242  2.12401015  3.07138726  1.9061834 ]
R^2: -0.0024651935487560728


In [179]:
reg_index.append("ElasticNet")
reg_results.append(dict(zip(reg_keys, [elastic_score, elastic_intercept] + elastic_coef.tolist())))

# 2. Classification

The following are a set of methods intended for classification in which the target value, $\hat{y}$,  is expected to be a linear combination of the $p$ features $x_i$:

$$\hat{y}(w, x) = f(w_0 + w_1 x_1 + ... + w_p x_p)$$  
where $w = (w_1,..., w_p)$ are the `coef_`, $w_0$ the `intercept_` and $f:\mathbb{R}\rightarrow[0;1]$

In [180]:
# Store intercept, weights and score metric to compare models

clf_keys = ["Class", "Accuracy", "intercept"] + [f"w{i}" for i in range(digits_X_train.shape[1])]
clf_results = []
clf_index = []

### 2.1 Logistic Regression

`LogisticRegression`, despite its name, is a linear model for classification rather than regression. Logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier. In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function. <br>
`LogisticRegression` is the equivalent of linear regression for the classification case, where we use the sigmoïd function:

$$f(X) = \frac{1}{1+ e^{-X}}$$  

<img src="../resources/img/logistic_function.png" width="450">


This implementation can fit binary, One-vs-Rest, or multinomial logistic regression with optional $\ell_1$, $\ell_2$ or Elastic-Net regularization. <br>
Regularization is applied by default, which is common in machine learning but not in statistics. Another advantage of regularization is that it improves numerical stability. No regularization amounts to setting C to a very high value. <br>
As an optimization problem, binary class $\ell_2$ penalized logistic regression minimizes the following cost function:
$$ \min_{w, c} \frac{1}{2}w^T w + C \sum_{i=1}^n \log(\exp(- y_i (X_i^T w + c)) + 1) .$$
Similarly, $\ell_1$ regularized logistic regression solves the following optimization problem:
$$\min_{w, c} \|w\|_1 + C \sum_{i=1}^n \log(\exp(- y_i (X_i^T w + c)) + 1).$$
**Elastic-Net** regularization is a combination of $\ell_1$ and $\ell_2$ and minimizes the following cost function:
$$\min_{w, c} \frac{1 - \rho}{2}w^T w + \rho \|w\|_1 + C \sum_{i=1}^n \log(\exp(- y_i (X_i^T w + c)) + 1)$$
where $\rho$ controls the strength of $\ell_1$ regularization vs. $\ell_2$ regularization (it corresponds to the `l1_ratio` parameter).

Note that, in this notation, it’s assumed that the target $y_i$ takes values in the set $[-1;1]$. We can also see that Elastic-Net is equivalent to $\ell_1$ when $\rho=1$ and equivalent to $\ell_2$ when $\rho=0$.

A logistic regression with $\ell_1$ penalty yields sparse models, and can thus be used to perform feature selection, as detailed in **L1-based feature selection**.

It is possible to obtain the p-values and confidence intervals for coefficients in cases of regression without penalization. The `statsmodels` package natively supports this. Within sklearn, one could use bootstrapping instead as well.

<img src="../resources/img/ols_vs_logistic_regression.png" width="650">

In [181]:
lr = LogisticRegression(max_iter=10000)
lr.fit(digits_X_train, digits_y_train)

lr_coef = lr.coef_
lr_intercept = lr.intercept_

lr_score = lr.score(digits_X_test, digits_y_test)

print(f"Accuracy: {lr_score}")

LogisticRegression(max_iter=10000)

Accuracy: 0.9722222222222222


In [182]:
clf_index.extend(["LogisticRegression"] * 10)
for i in range(10):
    clf_results.append(dict(zip(clf_keys, [i, lr_score, lr_intercept[i]] + lr_coef.tolist()[i])))

### 2.2 Perceptron

The *Perceptron* is another simple classification algorithm suitable for large scale learning. By default:

It does not require a learning rate.

It is not regularized (penalized).

It updates its model only on mistakes.

The last characteristic implies that the Perceptron is slightly faster to train than SGD with the hinge loss and that the resulting models are sparser.

The idea is similar to the Logistic Regression, but it's possible to use different activation function than simoïd, like ReLu, or hyperbolic tangent.

<img src="../resources/img/perceptron.png" width="650">

In [183]:
perceptron = Perceptron()
perceptron.fit(digits_X_train, digits_y_train)

perceptron_coef = perceptron.coef_
perceptron_intercept = perceptron.intercept_

perceptron_score = perceptron.score(digits_X_test, digits_y_test)

print(f"Accuracy: {perceptron_score}")

Perceptron()

Accuracy: 0.9527777777777777


In [184]:
clf_index.extend(["Perceptron"] * 10)
for i in range(10):
    clf_results.append(dict(zip(clf_keys, [i, perceptron_score, perceptron_intercept[i]] + perceptron_coef.tolist()[i])))

# 3. Bonus

## 3.1 Polynomial regression: extending linear models with basis functions

One common pattern within machine learning is to use linear models trained on nonlinear functions of the data. This approach maintains the generally fast performance of linear methods, while allowing them to fit a much wider range of data.

For example, a simple linear regression can be extended by constructing polynomial features from the coefficients. In the standard linear regression case, you might have a model that looks like this for two-dimensional data:
$$\hat{y}(w, x) = w_0 + w_1 x_1 + w_2 x_2$$
If we want to fit a paraboloid to the data instead of a plane, we can combine the features in second-order polynomials, so that the model looks like this:
$$\hat{y}(w, x) = w_0 + w_1 x_1 + w_2 x_2 + w_3 x_1 x_2 + w_4 x_1^2 + w_5 x_2^2$$
The (sometimes surprising) observation is that this is still a linear model: to see this, imagine creating a new set of features
$$z = [x_1, x_2, x_1 x_2, x_1^2, x_2^2]$$
With this re-labeling of the data, our problem can be written
$$\hat{y}(w, z) = w_0 + w_1 z_1 + w_2 z_2 + w_3 z_3 + w_4 z_4 + w_5 z_5$$
We see that the resulting polynomial regression is in the same class of linear models we considered above (i.e. the model is linear in ) and can be solved by the same techniques. By considering linear fits within a higher-dimensional space built with these basis functions, the model has the flexibility to fit a much broader range of data.

Here is an example of applying this idea to one-dimensional data, using polynomial features of varying degrees:

<img src="../resources/img/polynomial.png" width="400">

In [186]:
ols = LinearRegression()
polynomial_features = PolynomialFeatures(degree=2)

diabetes_polynomial_X_train = polynomial_features.fit_transform(diabetes_X_train)
diabetes_polynomial_X_test = polynomial_features.transform(diabetes_X_test)
ols.fit(diabetes_polynomial_X_train, diabetes_y_train)
 
ols_coef = ols.coef_
ols_intercept = ols.intercept_
 
ols_score = ols.score(diabetes_polynomial_X_test, diabetes_y_test)
 
print(f"Intercept: {ols_intercept}")
print(f"Coefficients: {ols_coef}")
print(f"R^2: {ols_score}")

LinearRegression()

Intercept: 65.35122267783325
Coefficients: [ 2.72265831e-09  1.08800751e+02 -3.11877460e+02  4.21938205e+02
  3.91310073e+02 -1.66531111e+04  1.45163271e+04  5.93019133e+03
  4.42673795e+01  6.13001899e+03  9.88062573e+00  2.38937018e+03
  1.52485368e+03 -1.29032694e+03  6.62502189e+02 -9.43431456e+02
 -7.24101051e+03  5.90997706e+03  9.79932135e+03  1.22536527e+03
  8.60270079e+02 -1.88326650e+00  3.45951367e+02  1.25466568e+03
  1.61897732e+03  1.64705649e+03 -3.53392670e+03 -7.60359401e+03
  2.12789918e+03  6.87728645e+02  8.01383901e+02  3.34807102e+03
 -6.15360849e+03  7.29631034e+03 -2.12219897e+02 -3.58505784e+03
  3.69391835e+03  4.74707827e+02 -2.56238551e+02  1.99355292e+04
 -1.46430010e+04 -8.28659726e+03 -1.39243467e+03 -7.89699555e+03
 -3.30251051e+03  9.38240672e+04 -1.29517278e+05 -6.18814888e+04
 -2.52509462e+04 -4.18286890e+04 -7.83704644e+03  4.60161861e+04
  3.47403128e+04  6.47107152e+03  2.50104208e+04  4.13987760e+03
  1.10995735e+04  1.61835840e+04  1.38485452e+0

In [193]:
diabetes_polynomial_X_train

array([[ 1.00000000e+00,  7.07687525e-02,  5.06801187e-02, ...,
         7.48991416e-04, -2.94941100e-05,  1.16143190e-06],
       [ 1.00000000e+00, -9.14709343e-03,  5.06801187e-02, ...,
         7.37045005e-08,  5.33003506e-06,  3.85448292e-04],
       [ 1.00000000e+00,  5.38306037e-03, -4.46416365e-02, ...,
         2.90263881e-04, -2.30069747e-04,  1.82358509e-04],
       ...,
       [ 1.00000000e+00,  3.08108295e-02, -4.46416365e-02, ...,
         1.18906721e-04,  1.17516833e-05,  1.16143190e-06],
       [ 1.00000000e+00, -1.27796319e-02, -4.46416365e-02, ...,
         1.47910332e-03,  1.47516308e-03,  1.47123335e-03],
       [ 1.00000000e+00, -9.26954778e-02, -4.46416365e-02, ...,
         2.64741942e-05,  5.54508555e-06,  1.16143190e-06]])

In [194]:
diabetes_X_train

array([[ 0.07076875,  0.05068012,  0.01211685, ...,  0.03430886,
         0.02736771, -0.0010777 ],
       [-0.00914709,  0.05068012, -0.01806189, ...,  0.07120998,
         0.00027149,  0.01963284],
       [ 0.00538306, -0.04464164,  0.04984027, ..., -0.00259226,
         0.01703713, -0.01350402],
       ...,
       [ 0.03081083, -0.04464164, -0.02021751, ..., -0.03949338,
        -0.01090444, -0.0010777 ],
       [-0.01277963, -0.04464164, -0.02345095, ..., -0.00259226,
        -0.03845911, -0.03835666],
       [-0.09269548, -0.04464164,  0.02828403, ..., -0.03949338,
        -0.00514531, -0.0010777 ]])

In [187]:
reg_index.append("PolynomialOLS")
reg_results.append(dict(zip(reg_keys, [ols_score, ols_intercept] + [None]*(len(reg_keys)-2))))

In some cases it’s not necessary to include higher powers of any single feature, but only the so-called interaction features that multiply together features (with the setting `interaction_only=True`). <br> 
For example, it's very usefull when dealing with boolean features.

In [188]:
perceptron = Perceptron()
polynomial_features = PolynomialFeatures(interaction_only=True)

digits_polynomial_X_train = polynomial_features.fit_transform(digits_X_train)
digits_polynomial_X_test = polynomial_features.transform(digits_X_test)
perceptron.fit(digits_polynomial_X_train, digits_y_train)

perceptron_score = perceptron.score(digits_polynomial_X_test, digits_y_test)

print(f"Accuracy: {perceptron_score}")

Perceptron()

Accuracy: 0.9861111111111112


In [189]:
clf_index.extend(["PolynomialPerceptron"] * 10)
for i in range(10):
    clf_results.append(dict(zip(clf_keys, [i, perceptron_score, perceptron_intercept[i]] + [None])))

# Results

In [190]:
pd.DataFrame(reg_results, index=reg_index)

,R^2,intercept,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9
OLS,0.452607,151.345655,37.900314,-241.966248,542.425753,347.708305,-931.461261,518.044055,163.403535,275.310038,736.189098,48.671125
Lasso,0.471855,151.574883,0.000000,-152.667066,552.694172,303.370551,-81.364834,-0.000000,-229.258298,0.000000,447.918189,29.642354
ElasticNet,-0.002465,153.717751,0.401004,0.000000,3.413534,2.326173,0.459577,0.125552,-1.783112,2.124010,3.071387,1.906183
PolynomialOLS,0.415688,65.351223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
pd.DataFrame(clf_results, index=clf_index).set_index("Class", append=True)

Accuracy  intercept   w0         w1          w2  \
                     Class                                                    
LogisticRegression   0      0.972222   3.476225  0.0  -0.005926   -0.008302   
                     1      0.972222  -6.072368  0.0   0.003930   -0.045444   
                     2      0.972222   0.758016  0.0   0.049614    0.031057   
                     3      0.972222  -1.762257  0.0   0.176249   -0.065441   
                     4      0.972222  13.744094  0.0  -0.023491   -0.224266   
                     5      0.972222  -9.415682  0.0   0.099814    0.544587   
                     6      0.972222   1.148184  0.0  -0.003099   -0.189879   
                     7      0.972222   5.038998  0.0   0.033279    0.059146   
                     8      0.972222   2.066906  0.0  -0.175131    0.061781   
                     9      0.972222  -8.982116  0.0  -0.155238   -0.163240   
Perceptron           0      0.952778  -4.000000  0.0  -9.000000  -19.000000   
                     1      0.952778 -93.000000  0.0 -58.000000    7.000000   
                     2      0.952778  -5.000000  0.0   4.000000  -15.000000   
                     3      0.952778  -8.000000  0.0  18.000000 -109.000000   
                     4      0.952778   3.000000  0.0  -7.000000  -60.000000   
                     5      0.952778 -23.000000  0.0  10.000000  243.000000   
                     6      0.952778 -14.000000  0.0 -14.000000 -172.000000   
                     7      0.952778  -3.000000  0.0 -26.000000  -13.000000   
                     8      0.952778 -72.000000  0.0 -49.000000   26.000000   
                     9      0.952778 -38.000000  0.0 -81.000000 -106.000000   
PolynomialPerceptron 0      0.986111  -4.000000  NaN        NaN         NaN   
                     1      0.986111 -93.000000  NaN        NaN         NaN   
                     2      0.986111  -5.000000  NaN        NaN         NaN   
                     3      0.986111  -8.000000  NaN        NaN         NaN   
                     4      0.986111   3.000000  NaN        NaN         NaN   
                     5      0.986111 -23.000000  NaN        NaN         NaN   
                     6      0.986111 -14.000000  NaN        NaN         NaN   
                     7      0.986111  -3.000000  NaN        NaN         NaN   
                     8      0.986111 -72.000000  NaN        NaN         NaN   
                     9      0.986111 -38.000000  NaN        NaN         NaN   

                                    w3          w4          w5          w6  \
                     Class                                                   
LogisticRegression   0        0.057673    0.012892   -0.074604   -0.132522   
                     1        0.164975   -0.632388    0.398091    0.027825   
                     2       -0.039294    0.123426   -0.029774    0.012596   
                     3        0.012411    0.461837    0.107177   -0.021714   
                     4       -0.419046   -0.290354   -0.368373   -0.246568   
                     5        0.046541    0.055948    0.196507    0.346433   
                     6       -0.136801   -0.014092   -0.041038   -0.037365   
                     7        0.082875    0.070952   -0.027308    0.182514   
                     8       -0.239287    0.114393   -0.116689   -0.236665   
                     9        0.469953    0.097386   -0.043988    0.105466   
Perceptron           0       27.000000  -65.000000  -80.000000  -25.000000   
                     1      108.000000 -446.000000  280.000000  -85.000000   
                     2      -29.000000   -8.000000  -54.000000  -46.000000   
                     3       42.000000  161.000000   12.000000  -42.000000   
                     4     -176.000000  -88.000000 -148.000000 -112.000000   
                     5      -57.000000  -84.000000  112.000000  251.000000   
                     6      -84.000000    6.000000  -39.000000  -63.000000   
                 